# Edge Generation using DexiNED

***

**Author:** Shane Cooke

**Date:** 3 Dec 2022

**Description:** DexiNED finds the edges of the images in 'data' folder and saves the edge maps in 'results' folder
***

## Imports

#### Install Dependencies

In [36]:
#!pip install kornia

#### Imports

In [29]:
import kornia
import os
import cv2
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np

***
## Edge Generation

In [32]:
!rm -rf ./DexiNed/data
!mkdir ./DexiNed/data

!rm -rf ./DexiNed/result
!mkdir ./DexiNed/result

!rm -rf ./Dataset/ASL_EDGE_MAP/Val
!mkdir ./Dataset/ASL_EDGE_MAP/Val

In [36]:
!cp -a ./Dataset/Augments/CROP/Val/. ./DexiNed/data/

In [37]:
%cd DexiNed

/home/shane/Documents/MastersCode/DexiNed


In [39]:
!python main.py

Number of GPU's available: 1
Pytorch version: 1.12.1
mean_bgr: [103.939, 116.779, 123.68]
/home/shane/anaconda3/envs/masters/lib/python3.10/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
output_dir: result/BIPED2CLASSIC
Restoring weights from: checkpoints/BIPED/10/10_model.pth
actual size: (262, 176, 3), target size: (512, 512)
actual size: (214, 203, 3), target size: (512, 512)
actual size: (209, 199, 3), target size: (512, 512)
actual size: (201, 235, 3), target size: (512, 512)
actual size: (119, 88, 3), target size: (512, 512)
actual size: (37, 23, 3), target size: (512, 512)
actual siz

#### Move Output Edge Maps into Dataset Folder

In [40]:
%cd ..
!cp -a ./DexiNed/result/BIPED2CLASSIC/avg/. ./Dataset/ASL_EDGE_MAP/Val

/home/shane/Documents/MastersCode


***
## Edge Map Augmentation

#### Create Paths

In [41]:
dataset = 'Val'

input_image_path = f'./Dataset/Augments/CROP/{dataset}/'
input_edge_path = f"./Dataset/ASL_EDGE_MAP/{dataset}/"

output_overlay_path = f"./Dataset/Augments/EDGE_B/{dataset}/"
os.makedirs(output_overlay_path,exist_ok=True)

output_thinned_overlay_path = f"./Dataset/Augments/EDGE_B/THIN/{dataset}/"
os.makedirs(output_thinned_overlay_path,exist_ok=True)

for filename in os.listdir(input_image_path):
    if filename.endswith('.JPG'):
        os.rename(os.path.join(input_image_path, filename), os.path.join(input_image_path, filename[:-3] + "jpg"))

jpg_files = [pos_jpg for pos_jpg in os.listdir(input_image_path) if pos_jpg.endswith('.jpg')]

#### Required Functions

In [42]:
def overlay(image, edge, directory, name):
    
    #edge[np.all(edge == (255,255,255), axis=-1)] = (255,255,0)
    edge = cv2.bitwise_not(edge)
    edge_array = Image.fromarray(edge)
    edge_rgba = edge_array.convert("RGBA")
    datas = edge_rgba.getdata()

    newData = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((0, 0, 0, 0))
        else:
            newData.append(item)

    edge_rgba.putdata(newData)
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image_array = Image.fromarray(image_bgr)
    image_array.paste(edge_rgba, (0, 0), edge_rgba)
    image_array.save(f'{directory}{name}')

In [43]:
def thin_edge(edge):
    gray_edge = cv2.cvtColor(edge, cv2.COLOR_BGR2GRAY)
    gray_edge_copy = gray_edge.copy()

    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
    thinned_edge = np.zeros(gray_edge_copy.shape, dtype='uint8')

    while(cv2.countNonZero(gray_edge_copy) != 0):
        erode = cv2.erode(gray_edge_copy, kernel)
        opening = cv2.morphologyEx(erode, cv2.MORPH_OPEN, kernel)
        subset = erode - opening
        thinned_edge = cv2.bitwise_or(subset, thinned_edge)
        gray_edge_copy = erode.copy()

    thinned_edge_bgr = cv2.cvtColor(thinned_edge, cv2.COLOR_GRAY2BGR)

    return thinned_edge_bgr

#### Convert to binary, overlay, thin & overlay

In [44]:
for jpg_image in jpg_files:
    
    png_image = jpg_image.replace(".jpg",".png")
    jpg_image_read = cv2.imread(f'{input_image_path}{jpg_image}')
    png_image_read = cv2.imread(f'{input_edge_path}{png_image}')

    inverted_image = cv2.bitwise_not(png_image_read)

    ret, binary = cv2.threshold(inverted_image, 100, 255, cv2.THRESH_BINARY)
    bw = cv2.threshold(inverted_image, 100, 255, cv2.THRESH_BINARY)

    overlay(jpg_image_read, binary, output_overlay_path, jpg_image)

    thinned_edge_bgr = thin_edge(binary)

    overlay(jpg_image_read, thinned_edge_bgr, output_thinned_overlay_path, jpg_image)